# Práctica 2: Búsquedas no informadas
### [Introducción a los sistemas inteligentes 2019-1](https://fagonzalezo.github.io/iis-2019-1/)

___________

**Fecha límite de entrega**: _Miércoles 24 de Abril_ antes de la medianoche. 
Envios fuera de este plazo no se tendrán en cuenta.


Cerciórese de reiniciar y correr el notebook en su totalidad antes de enviarlo. Verifique que todas las salidas se muestran de manera correcta.

Integrantes del grupo (máximo 3):

* Samael de Jesus Salcedo Amortegui 1016099109
* Andrés Fernando Román Arévalo 1007098201
* German David Garcia Nieves 1026303730

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/SLNaSv6au3PvQruWkxMO) antes de la medianoche de la fecha límite. El archivo debe nombrarse como  isi-practica2-unalusername1-unalusername2-unalusername3.ipynb, donde unalusername es el nombre de usuario asignado por la universidad (incluya los nombres de usuario de todos los miembros del grupo).

-----


## 1. El problema de las jarras de agua

Usted tiene 2 jarras, una con capacidad de 3 litros y otra con capacidad de 5 litros. Además, se
cuenta con un grifo. Las jarras se pueden llenar totalmente, vaciar en el piso y vaciar parcialmente el
contenido de una jarra en la otra. Las jarras no tienen ninguna clase de marca. El objetivo es medir
exactamente un litro de agua.

### 1.1 
Modele este problema creando una clase que herede de siguiente clase abstracta:




# Main Class Problem


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    
    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

# Queues

In [0]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Pour Problem
Problema modelado herado de la clase problem :V


In [0]:
class PourProblem(Problem):
    
    def actions(self, state):
        jugs = range(len(state))
        return ([('Fill', i)    for i in jugs if state[i] < self.sizes[i]] +
                [('Dump', i)    for i in jugs if state[i]] +
                [('Pour', i, j) for i in jugs if state[i] for j in jugs if i != j])

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        result = list(state)
        act, i, *_ = action
        
        if act == 'Fill':   # Fill i to capacity
            result[i] = self.sizes[i]
        
        elif act == 'Dump': # Empty i
            result[i] = 0
        
        elif act == 'Pour': # Pour from i into j
            j = action[2]
            amount = min(state[i], self.sizes[j] - state[j])
            result[i] -= amount
            result[j] += amount
        return tuple(result)

    def is_goal(self, state):
        """True if the goal level is in any one of the jugs."""
        return self.goal in state

**Nota:** Utilice como guía este [notebook](https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) del AIMA Github Repository donde se implementa el problema para un número arbitrario de jarras. 

### 1.2 
Encuentre una solución utilizando búsqueda en amplitud. Utilice la función `breadth_first_search` en el [notebook](https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) de AIMA. Su código debe imprimir cada uno de los pasos de la solución. 

**Nota**: la función `breadth_first_search` devuelve un nodo del árbol de búsqueda. Busque la funciones en el notebook que le permiten obtener la secuencia de acciones de la solución a partir del nodo resultante. 

# Breadth First Search

In [0]:
def breadth_first_search(problem):
    
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached and s != None:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [5]:
p1 = PourProblem((0,0), 1, sizes=(5,3))
soln = breadth_first_search(p1)
path_actions(soln), path_states(soln)


([('Fill', 1), ('Pour', 1, 0), ('Fill', 1), ('Pour', 1, 0)],
 [(0, 0), (0, 3), (3, 0), (3, 3), (5, 1)])

### 1.3
Encuentre una solución utilizando búsqueda en profundidad. Utilice la función `depth_first_recursive_search` en el  [notebook](https://github.com/aimacode/aima-python/blob/master/search4e.ipynb) de AIMA. Su código debe imprimir cada uno de los pasos de la solución. 

¿La solución difiere de la solución producida por BFS? Explique.

# Depth First Recursive Search 

In [0]:
def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)
  
def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result:
                return result
        return failure

In [0]:
dfs = depth_first_recursive_search(p1)

In [8]:

path_states(dfs)


[(0, 0), (5, 0), (5, 3), (0, 3), (3, 0), (3, 3), (5, 1)]

In [9]:
path_actions(dfs)

[('Fill', 0),
 ('Fill', 1),
 ('Dump', 0),
 ('Pour', 1, 0),
 ('Fill', 1),
 ('Pour', 1, 0)]



** Al ejecutar BFS se encuentra la solucion mas optima(la que tiene menos acciones a realizar) mientras que DFS al hacer la busqueda por profundidad encuentra una solucion no necesariamente la mas optima**





## 2. El problema de los misioneros y los caníbales (MC).

Tres misioneros y tres caníbales deben cruzar un río usando un bote que puede llevar como máximo a dos personas, bajo la restricción de que, para ambos lados del río, si hay misioneros presentes en un lado, no pueden ser superados en número por caníbales (si lo fueran, los caníbales se comerían a los misioneros). El barco no puede cruzar el río por sí mismo sin gente a bordo.

Su objetivo es modelar este problema como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

### 2.1 (2.0) Cree una clase para modelar el problema MC

Un estado del problema puede representarse de diferentes formas. La sugerencia es usar una tupla de valores indicando el número de misioneros y caníbales en cada lado del rio, así como la posición del bote:

```python
('I', 3, 2, 0, 1)
```

Representa un estado en el cual el bote se encuentra en el lado izquierdo, hay 3 misioneros en el lado izquierdo, 2 caníbales en el lado izquierdo, cero misioneros en el lado derecho y 1 canibal en el lado derecho.

Una acción puede representarse como una pareja de valores indicando cuantos misioneros y caníbales se van a mover. Por ejemplo el siguiente vector

```python
(0, 1)
```

Indica que se moverán cero misioneros y un canibal. Note que no es necesario representar el bote, pues este siempre se mueve.

Asuma que al principio todos los actores están a la izquierda y al final todos deben estar a la derecha.



# Misioneros y canibales



In [0]:
class RiverProblem(Problem):
    
    def actions(self, state):
        return [(0,1),(1,0),(1,1),(2,0),(0,2)]
        

    def result(self, state, action):
        
        result = list(state) 
        stat = list(action) 
        # misionesros - canibal
        
        if stat[0]==1 and stat[1]==1:
          if (result[1]>=result[2] and result[3]>= result[4]) or (result[3]==0 and result[1]>=result[2]) or (result[1]==0 and result[3]>=result[4]):
            if result[0]=="L":
              result[0]="R"
              result[1]=result[1]-stat[0]
              result[2]=result[2]-stat[1]
              result[3]=result[3]+stat[0]
              result[4]=result[4]+stat[1]
            else:
              result[0]="L"
              result[1]=result[1]+stat[0]
              result[2]=result[2]+stat[1]
              result[3]=result[3]-stat[0]
              result[4]=result[4]-stat[1]
            
        #Paso de un o dos misionero
        elif stat[0]==1 or stat[0]==2:
          if (result[1]>=result[2] and result[3]>= result[4]) or (result[3]==0 and result[1]>=result[2]) or (result[1]==0 and result[3]>=result[4]):
            if result[0]=="L":
              result[0]="R"
              result[1]=result[1]-stat[0]
              result[3]=result[3]+stat[0]
            else:
              result[0]="L"
              result[1]=result[1]+stat[0]
              result[3]=result[3]-stat[0]
        
        #Paso de un o dos canibales
        elif stat[1]==1 or stat[1]==2:
          if(result[1]>=result[2] and result[3]>= result[4]) or (result[3]==0 and result[1]>=result[2]) or (result[1]==0 and result[3]>=result[4]):
            if result[0]=="L":
              result[0]="R"
              result[2]=result[2]-stat[1]
              result[4]=result[4]+stat[1]
            else:
              result[0]="L"
              result[2]=result[2]+stat[1]
              result[4]=result[4]-stat[1]
        if 0<=result[1]<=3 and 0<=result[2]<=3 and 0<=result[3]<=3 and 0<=result[4]<=3:
          return tuple(result)
        else:
          return None

    def is_goal(self, state):
        return self.goal == state

In [0]:
r1 = RiverProblem(('L', 3, 3, 0, 0), ('R', 0 , 0 , 3 , 3))

### 2.2   Use un método de búsqueda para encontrar una solución óptima al problema

Cree una función que calcule la secuencia de acciones que resuelva el problema así como los estados (incluidos el inicial y el final) que se  visitan al ejecutar la solución. Cuántos movimientos requiere la solución óptima?

In [0]:
bfs_river = breadth_first_search(r1)

11 movimientos se requieren para la solución óptima

#Acciones

In [13]:
path_actions(bfs_river)

[(1, 1),
 (1, 0),
 (0, 2),
 (0, 1),
 (2, 0),
 (1, 1),
 (2, 0),
 (0, 1),
 (0, 2),
 (0, 1),
 (0, 2)]

# Estados desde inicial hasta el final


In [14]:
path_states(bfs_river)

[('L', 3, 3, 0, 0),
 ('R', 2, 2, 1, 1),
 ('L', 3, 2, 0, 1),
 ('R', 3, 0, 0, 3),
 ('L', 3, 1, 0, 2),
 ('R', 1, 1, 2, 2),
 ('L', 2, 2, 1, 1),
 ('R', 0, 2, 3, 1),
 ('L', 0, 3, 3, 0),
 ('R', 0, 1, 3, 2),
 ('L', 0, 2, 3, 1),
 ('R', 0, 0, 3, 3)]

### 2.3  Estados alcanzables desde el estado inicial

Cree una función que calcule todos los estados alcanzables desde el estado inicial, es decir estados para los que existe una secuencia de acciones que lleva del estado inicial a ellos. ¿Son todos los estados posibles alcanzables?

In [0]:
Nodos = list()

def bfs(problem):
    node = Node(problem.initial)
    Nodos.append(list(problem.initial))
    
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                Nodos.append(list(s))
                return child
            if s not in reached and s != None:
                
                if s not in Nodos:
                    Nodos.append(list(s))
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [0]:
def Expand():
    bfs(r1)
    h = 0
    for i in range(len(Nodos)):
        p = Nodos[i]
        
        if p[1] == 0:
            if p[3] >= p[4]:
                print(p) 
                h = h + 1
      
        elif p[3] == 0:
            if p[1] >= p[2]:
                print(p) 
                h = h + 1
        else: 
            if p[1] >= p[2] and p[3] >= p[4]:
                print(p) 
                h= h + 1
      
    print("Numero de nodos Alcanzables: ",h)

In [17]:
Nodos = list()
Expand()

['L', 3, 3, 0, 0]
['R', 3, 2, 0, 1]
['R', 2, 2, 1, 1]
['R', 3, 1, 0, 2]
['L', 3, 2, 0, 1]
['R', 3, 0, 0, 3]
['L', 3, 1, 0, 2]
['R', 1, 1, 2, 2]
['L', 2, 2, 1, 1]
['R', 0, 2, 3, 1]
['L', 0, 3, 3, 0]
['R', 0, 1, 3, 2]
['L', 0, 2, 3, 1]
['L', 1, 1, 2, 2]
['R', 0, 0, 3, 3]
Numero de nodos Alcanzables:  15


**Desde el estado inicial todos los posibles estados no son alcanzables dado que, en algunos de ellos no cumplen las condiciones iniciales para que fueran validos**

Ejemplo: que hayan mas canibales que misioneros en alguno de los dos lados o en ambos, Tambien que se quieran mover misioneros o canibales de el lado derecho o izquierdo cuando en dicho lado no hay ninguno.